In [20]:
from opentrons_json import plates,commands
import opentrons_json
import datetime
import uuid
import json

with open('./../protocol-schema.json') as f:
    data = json.load(f)
from jsonschema import validate


with open('/home/koeng/Downloads/response_1554406909932.json') as f:
    test_plate = json.load(f)

opentrons = {'uuid': '43346aba-41e0-483d-ad43-d46a85a6ead5',
             'right_300': '9c3b3205-4fe3-4bf1-af3f-8cf9ab95d918',
             'left_10': '82e95a7a-f335-4178-a7b6-4f38954e497a'
            }

In [158]:
import string
class Plate():
    def __init__(self, height:int=8, length:int=12,locations:list=[]) -> None:
        positions = []
        for letter in list(string.ascii_uppercase[0:height]):
            for number in range(length):
                positions.append((letter, number+1))
        self.height = height
        self.length = length
        if locations==[]:
            self.locations = [x[0]+str(x[1]) for x in positions]
        else:
            self.locations = locations

    def rows(self)-> list:
        return [list(g) for k, g in groupby(self.locations, key=lambda x: x[0])]
    def columns(self) -> list:
        return [[x for x in self.locations if x[1:] == str(num+1)] for num in range(self.length)]
    def reverse_columns(self)-> list:
        return [sorted(x, reverse=True) for x in self.columns()]
    def flat_columns(self) -> list:
        return [item for sublist in self.columns() for item in sublist]


    def subcolumns(self)-> list:
        if self.height%2 != 0:
            raise StandardError('Not compatible with multichannel')
        spaces = int(self.height/8)
        subcolumns=[]
        for column in self.columns():
            for space in range(spaces):
                count = space
                subcolumn=[]
                while count < len(column):
                    subcolumn.append(column[count])
                    count+=spaces
                subcolumns.append(subcolumn)
        return subcolumns
    
    def use_wells(self,tips:list)-> str:
        self.locations = [v for i, v in enumerate(self.locations) if v not in tips]
        return tips

    def use_number(self,num_wells:int=8,columns=None) -> str:
        if columns == None:
            columns=self.reverse_columns()
        for column in columns:
            if len(column) >= num_wells:
                return self.use_wells(column[0:num_wells])
        raise ValueError("No more tips.")

def data_transfer_comprehension(data_transfer, old_plate=Plate(),new_plate=Plate()):
    output=[]
    for transfer in data_transfer:
        original_index = old_plate.flat_columns().index(transfer[1])
        new_index = new_plate.flat_columns().index(transfer[2])
        for i in range(transfer[0]):
            old_well = old_plate.flat_columns()[i+original_index]
            new_well = new_plate.flat_columns()[i+new_index]
            output.append((old_well,new_well))
    return output
            

def efficient_transfer(derived,full=Plate().subcolumns()):
    derived = Plate(locations=derived).subcolumns()
    transfers=[]
    well = None
    for index,subcolumn in enumerate(derived):
        tip_count = 1
        loc_num=0
        wells=[]
        counter = 0
        for well_index,well in enumerate(subcolumn):

            # Check if this is a new well
            wells.append(well)

            # Check that this is not the last location we can pull from 
            if not well_index+2 > len(subcolumn):
                loc_num = full[index].index(subcolumn[well_index])-well_index-counter



            # If last location, add tips and location, reset new well
            if loc_num != 0:
                if loc_num != 50:
                    transfers.append((len(wells[:-1]),wells[:-1]))
                    wells=[wells[-1]]
                    tip_count=0
                    counter=loc_num
                else:
                    transfers.append((len(wells),wells))
                    wells=[]
                    tip_count=0
                    counter=loc_num
            
            loc_num=50
            tip_count+=1
    return transfers

In [ ]:
def transfer(volume,from_wells,tipbox=Plate(),from_plate_layout=Plate(),to_plate_layout=Plate()):
    def efficient_tip(derived,full):
        transfers=[]
        well = None
        for index,subcolumn in enumerate(derived):
            tip_count = 1
            loc_num=0
            wells=[]
            for well_index,well in enumerate(subcolumn):

                # Check if this is a new well
                wells.append(well)

                # Check that this is not the last location we can pull from 
                if not well_index+2 > len(subcolumn):
                    loc_num = full[index].index(subcolumn[well_index+1])-well_index

                # If last location, add tips and location, reset new well
                if loc_num != 1:
                    transfers.append((tip_count,wells))
                    wells=[]
                    tip_count=0

                loc_num=0
                tip_count+=1
        return transfers
    
    transfers = efficient_tip(Plate(locations=from_wells).subcolumns(),from_plate_layout.subcolumns())
    to = []
    for step in transfers:
        tips = tipbox.use_number(step[0])
        to+=to_plate_layout.use_number(step[0],to_plate_layout.columns())
    transfer_from = [item for sublist in [step[1] for step in transfers] for item in sublist]
    
    individual_transfers = [(volume,) + transfer for transfer in list(zip(transfer_from,to))]
    return transfers,individual_transfers

In [168]:
class Protocol():
    def __init__(self,robot:dict,labware:dict,procedure:dict,protocol_schema:str='1.0.0',metadata:dict={},defaults:dict={},designer_application:dict={},pipettes:dict={}):
        # Metadata
        self.protocol_schema = protocol_schema
        self.metadata = metadata
        
        # Defaults
        if defaults == {}:
            self.defaults = opentrons_json.default_values
        else:
            self.defaults = defaults
        
        #Designer application
        if designer_application == {}:
            self.designer_application = opentrons_json.designer_application
        else:
            self.designer_application = designer_application
            
        # Pipettes 
        if pipettes == {}:
            self.pipettes = {
                robot['left_10']: opentrons_json.pipette("left", "p10_multi", "p10_multi_v1.3"),
                robot['right_300']: opentrons_json.pipette("right", "p300_multi", "p300_multi_v1.3")
            }
        else:
            self.pipettes = pipettes
        
        self.labware = labware
        self.procedure=procedure
            
    def toJSON(self):
         return {
             "protocol-schema": self.protocol_schema,
             "metadata": self.metadata,
             "default-values": self.defaults,
             "designer-application": self.designer_application,
             "robot": opentrons_json.ot2_standard,
             "pipettes": self.pipettes,
             "labware": self.labware,
             "procedure": self.procedure}
        
def transformation(robot,plate,breadcrumb=None,strain='E.coli Top10',transformant_volume=15,media='lb',transformation_volume=2):
    # Create new plate
    new_plate = {"plate_form": "96pcr",
                 "plate_name": "{}-transformation".format(plate['plate_name']),
                 "plate_type": "transformation",
                 "status": "planned",
                 "breadcrumb": breadcrumb,
                 "uuid": str(uuid.uuid4())}

    # Metadata
    metadata = {
            "protocolName": "Transformation of {}".format(plate['uuid']),
            "author": "Keoni Gandall",
            "datetime": "{}".format(str(datetime.datetime.now()))
          }

    # Addresses from
    addresses = []
    for well in plate['wells']:
        addresses.append(well['address'])

    # Tipbox
    tipbox = Plate()
    new = Plate()

    # Labware
    pipette = robot['left_10']
    tips = '11_tip_box'

    labware = {
        tips: opentrons_json.labware('11',"opentrons-tiprack-10ul",name="11_tip_box"),
        'trash_yo': opentrons_json.labware('12',"fixed-trash",name="Trash yo"),
        new_plate['uuid']: opentrons_json.labware('5', '96-PCR-flat', name=new_plate['plate_name']),
        plate['uuid']: opentrons_json.labware('4', '96-PCR-flat', name=plate['plate_name'])
    }
    
    # Create new procedure
    procedure = []
    data_transfer = []
    high_level_transfers = efficient_transfer(addresses)
    volume = transformation_volume
    for transfer in high_level_transfers:
        procedure.append(commands.pick_up_tip(pipette,tips,tipbox.use_number(transfer[0])[-1]))
        transfer_from = transfer[1][0]
        procedure.append(commands.aspirate(pipette,volume,plate['uuid'],transfer_from))
        transfer_to = new.use_number(transfer[0],columns=new.columns())[0]
        procedure.append(commands.dispense(pipette,volume,new_plate['uuid'],transfer_to))
        procedure.append(commands.drop_tip(pipette,'trash_yo','A1'))
        data_transfer.append((transfer[0],transfer_from,transfer_to))
    
    protocol = {'uuid': str(uuid.uuid4()),
               'description': metadata['protocolName'],
               'protocol': Protocol(robot,labware,[{'subprocedure': procedure}],metadata=metadata).toJSON(),
               'status': 'planned'}
    
    transfers = data_transfer_comprehension(data_transfer)
    new_wells=[]
    for transfer in transfers:
        old_well = {}
        for well in plate['wells']:
            if well['address'] == transfer[0]:
                old_well = well
        new_well = {'address': transfer[1],
                    'media': media,
                    'organism': strain,
                    'plate_uuid': new_plate['uuid'],
                    'quantity': None,
                    'samples': [sample['uuid'] for sample in old_well['samples']],
                    'volume': transformant_volume+transformation_volume,
                   'well_type': 'transformation'}
        new_wells.append(new_well)
    
    output = {'Plate': [new_plate],
             'Well': new_wells,
             'Protocol': [protocol]}
    return output
    
        

result = transformation(opentrons,test_plate)#['protocol']
#validate(result,schema=data)

In [169]:
result

{'Plate': [{'plate_form': '96pcr',
   'plate_name': '23w-transformation',
   'plate_type': 'transformation',
   'status': 'planned',
   'breadcrumb': None,
   'uuid': '8f98360e-923e-4e8f-bb31-6846d26d8492'}],
 'Well': [{'address': 'A1',
   'media': 'lb',
   'organism': 'E.coli Top10',
   'plate_uuid': '8f98360e-923e-4e8f-bb31-6846d26d8492',
   'quantity': None,
   'samples': ['d7f18ce0-b423-463d-9da4-058b2c1a88af'],
   'volume': 17,
   'well_type': 'transformation'},
  {'address': 'B1',
   'media': 'lb',
   'organism': 'E.coli Top10',
   'plate_uuid': '8f98360e-923e-4e8f-bb31-6846d26d8492',
   'quantity': None,
   'samples': ['f91cdf69-51c5-4e24-85d4-c4ecb64e32ed'],
   'volume': 17,
   'well_type': 'transformation'},
  {'address': 'C1',
   'media': 'lb',
   'organism': 'E.coli Top10',
   'plate_uuid': '8f98360e-923e-4e8f-bb31-6846d26d8492',
   'quantity': None,
   'samples': ['82521fca-29d1-4e85-97e6-756d2a2991a5'],
   'volume': 17,
   'well_type': 'transformation'},
  {'address': 'D1'

In [162]:
def new_wells_samples(old_plate,data_transfer,new_plate,strain,transformant_volume,transformation_volume,media):
    transfers = data_transfer_comprehension(data_transfer)
    for transfer in transfers:
        old_well = {}
        for well in old_plate['wells']:
            if well['address'] == transfer[0]:
                old_well = well
        new_well = {'address': transfer[1],
                    'media': media,
                    'organism': strain,
                    'plate_uuid': 'a364eab8-0c68-46b2-8aca-51b72d2f3cb5',
                    'quantity': None,
                    'samples': [sample['uuid'] for sample in old_well['samples']],
                    'volume': transformant_volume+transformation_volume,
                   'well_type': 'transformation'},



data_transfer_comprehension(result)

[('A1', 'A1'),
 ('C1', 'B1'),
 ('D1', 'C1'),
 ('E1', 'D1'),
 ('F1', 'E1'),
 ('G1', 'F1'),
 ('H1', 'G1'),
 ('A2', 'A2'),
 ('B2', 'B2'),
 ('C2', 'C2'),
 ('D2', 'D2'),
 ('E2', 'E2'),
 ('F2', 'F2'),
 ('G2', 'G2'),
 ('H2', 'H2'),
 ('A3', 'A3'),
 ('B3', 'B3'),
 ('C3', 'C3'),
 ('D3', 'D3'),
 ('E3', 'E3'),
 ('F3', 'F3'),
 ('G3', 'G3'),
 ('H3', 'H3'),
 ('A4', 'A4'),
 ('B4', 'B4'),
 ('C4', 'C4'),
 ('D4', 'D4'),
 ('E4', 'E4'),
 ('F4', 'F4'),
 ('G4', 'G4'),
 ('H4', 'H4'),
 ('A5', 'A5'),
 ('B5', 'B5'),
 ('C5', 'C5'),
 ('D5', 'D5'),
 ('E5', 'E5'),
 ('F5', 'F5'),
 ('G5', 'G5'),
 ('H5', 'H5'),
 ('A6', 'A6'),
 ('B6', 'B6'),
 ('C6', 'C6'),
 ('D6', 'D6'),
 ('E6', 'E6'),
 ('F6', 'F6'),
 ('G6', 'G6'),
 ('H6', 'H6'),
 ('A7', 'A7'),
 ('B7', 'B7'),
 ('C7', 'C7'),
 ('D7', 'D7'),
 ('E7', 'E7'),
 ('F7', 'F7'),
 ('G7', 'G7'),
 ('H7', 'H7'),
 ('A8', 'A8'),
 ('B8', 'B8'),
 ('C8', 'C8'),
 ('D8', 'D8'),
 ('E8', 'E8'),
 ('F8', 'F8'),
 ('G8', 'G8'),
 ('H8', 'H8'),
 ('A9', 'A9'),
 ('B9', 'B9'),
 ('C9', 'C9'),
 ('D9', 'D